# PreProcessing: Feature Selection

Feature Selection is an important step in data pre-processing. It consists in selecting the best subset of input variable as the most pertinent. Discarding irrelevant data is essential before applying Machine Learning algorithm in order to:
* *Reduce Overfitting*: less opportunity to make decisions based on noise;
* *Improve Accuracy*: less misleading data means modelling accuracy improves. Predictions can be greatly distorted by redundant attributes. 
* *Reduce Training Time*: With less data the algorithms will train faster;


### Configuration Notebook
Before running the notebook, check these parameter value in order to configure dataset and method option. In particular, we have:
- **DATA_NORMALIZED**: If *True* data will be normalized between (0,1) before feature selection;
- **DATA_STANDARDIZED**: If *True* data will be standarized before feature selection;
- **RESULTS_NORMALIZED**: If *True* results of feature selection will be normalized between (0,1);
- **TARGET_VARIABLE**: It's the target variable used for regression analysis.
- **NUMBER_OF_FEATURE_RFE**: It's number of features of the best subset obtained by *recursive feature selection method*;
- **DATASET_PATH**: path of the .gpkg file imported;
- **TH_VALUE** Threshold value used by variance_threshold method.

In [1]:
DATA_NORMALIZED = False
DATA_STANDARDIZED = True
RESULTS_NORMALIZED = True
DATASET_PATH = 'grids/grid_cams_42_43_2020.gpkg'
TARGET_VARIABLE = 'pm25_cams'
NUMBER_OF_FEATURE_RFE = 5
TH_VALUE=0.1

### Import Libraries

In [2]:
%matplotlib inline
import scipy.stats as stats
import geopandas as gpd
import warnings
import numpy as np
from fs import methods as m
import ipywidgets as widgets
import pandas as pd
warnings.filterwarnings("ignore")
from IPython.core.display import display

### Data import
Data are imported from .gpkg file, splitted in **train_set** and **test_set** which are used by supervised algorithm. Otherwise with unsupervised algorithm the entire dataset will be processed.

In [3]:
#read gpkg file
data = gpd.read_file(DATASET_PATH)
#read variables which are not null
labels = m.noSensor_features(data.columns.tolist())
score_results = pd.DataFrame()

### Definition of dataframe used for feature selection


In [4]:
X = m.clean_dataset_nosensor(data, labels)
Y = X[[TARGET_VARIABLE]]
Y = Y.values.ravel()
X.pop(TARGET_VARIABLE)
X.pop('bottom')
X.pop('top')
X.pop('left')
X.pop('right')


#coordinates definition used for mgwr
coords = list(zip(X['lat_cen'], X['lng_cen']))
X.pop('lat_cen')
X.pop('lng_cen')
score_results['Features'] = X.columns.tolist()
X_notStand = X

if(DATA_NORMALIZED):
    X_notNorm= X
    X = m.NormalizeData2D(X)

if(DATA_STANDARDIZED):
    X = X.apply(stats.zscore)

Due to the fact there isn’t a best feature selection technique, many different methods are performed. The aim of this part is to discover by experimentation which one/ones work better for this specific problem.
In this study, I choose supervised methods, which are classified into 3 groups, based on their  different approach.


## Filter Methods
Filter-based feature selection methods adopt statistical measures to evaluate the correlation/dependence between input variables.
These select features from the without machine learning algorithm. In terms of computation, they are very fast and are very suitable in order to remove duplicated, correlated, redundant variables. On the contrary,  these methods do not remove multicollinearity.


### Pearson correlation index
It's a measure of linear correlation which stands for the ratio between the covariance of two variables and the product of their standard deviations. The algorithm evaluate the index for each feature variable.

In [5]:
score_results['pearson'] = m.pearson(X, Y)

Output()

### Spearmanr correlation coefficient
The Spearman correlation coefficient is a measure of the monotonicity of the relationship between two datasets.
The algorithm evaluate the index for each feature variable.



In [6]:
score_results['spearmanr'] = m.spearmanr(X, Y)

Output()

### Kendall tau
As the Spearmanr correlation coefficient, is based on the ranks of data.
In most of the situations, the interpretations of Kendall’s tau and Spearman’s rank correlation coefficient are very similar and thus invariably lead to the same inferences.
The algorithm evaluate the index for each feature variable.


In [7]:
score_results['kendall'] = m.kendall(X, Y)


Output()

### Chi-square test not allowed with no categorical input
A chi-square test is used to test events independence. In feature selection instead, we aim to select the features which are more highly dependent. Even if is more suitable for categorical input, computation is performed by casting features values as *Int values*.

In [8]:
# = m.chi2_test(X, Y)
#score_frame.append(tuple)

### F-Test
It's a statistical test used to compare model from X and Y and check if the difference is significant between them through regression. The algorithm evaluate the Fisher score for each feature variable.

In [9]:
score_results['fisher'] = m.f_test(X, Y)


Output()

### Dispersion Ratio (Unsupervised)
It stands for the ratio between arithmetic mean the geometric mean. This is very usefull to check dispersion on data. Higher dispersion implies a higher value of this coefficent, thus a more relevant variable.

In [10]:

#results['dispersion_ratio'] = m.compute_dispersion_ratio(preprocessing.normalize(X))

### Variance Threshold 
It's an approach which aims to remove all features which variance doesn’t meet some threshold. Usually it removes all zero-variance features, so variables taht contains useless information.


In [11]:
score_results['variance_threshold'] = m.variance_threshold(X_notStand, TH_VALUE)

Output()

## Wrapper Methods (and Embedded)
Wrapper methods, as the name suggests, wrap a machine learning model, with different subsets of input features: In this way the subsets are  evaluated following  the best model performance.
Embedded methods instead are characterised by the benefits of both the wrapper and filter methods, by including interactions of features but also having a reasonable computational cost.


### Exhaustive feature selection for regression analysis
This algorithm follow the exhaustive feature selection approach with brute-force evaluation of feature subsets; the best subset is selected by optimizing a specified metric given an arbitrary regressor or classifier. In this case a transformer is used to perform the Sequential Feature Selection.
The final outputs are:
* Accuracy for the subset choosen;
* Indices of the choosen features;
* Corresponding names of the features choosen;



In [12]:
#m.exhaustive_feature_selection(X, Y)

### Random Forest importance
It uses a forest of trees to evaluate the importance of each features as outputs.

In [13]:
score_results['RF_importance'] = m.RF_importance(X, Y)


Output()

### Recursive Feature Selection
The goal of RFE is to select features by recursively considering smaller and smaller sets of features.
The final output for each variable will be:
* *Label*;
* A *boolean expressing* whetever is selected or not;
* *Ranking value* based on its score;

In [14]:
m.recursive_feature_selection(X, Y.astype(int), NUMBER_OF_FEATURE_RFE)

,Features,isSelected,Ranking
0,dusaf,False,85
1,dusafSum,False,84
2,dsf2,False,83
3,dsf3,False,81
4,dsf4,False,80
...,...,...,...
84,prec_int,False,34
85,rad_glob_int,False,40
86,temp_int,False,56
87,wind_dir_int,False,69


## Multiscale Geographically Weighted Regression
Due to the fact that this study is related to geographic and spatial data, each pieces of data is very sensitive to the geographic distance between them. So the use of mgwr methods could be innovative, since multivariate models are increasingly encountered in geographical research to estimate spatially varying relationships between a targets and its predictive variables.




In [15]:
#m.mgwr(data, list(X.columns), coords, Y)


from sklearn import preprocessing

## Results
Results before are collected in a Pandas Dataframe called **score_results**. In each column are shown the score results for each method; instead, each row collect the results for each feature variable.
* **Features**: Set of feature names (Strings);
* **pearson**:  Pearson correlation index scores;
* **spearmanr**: Spearmanr correlation coefficient scores;
* **kendall**: Kendall tau scores; 
* **fisher**: F-Test scores;
* **variance_threshold**: Variance Threshold  scores;
* **RF_importance**: Random Forest importance scores;

* **avg_scores**: scores obtained from the mean of the previous scores for each variables; this values are displayed through a bar plot.

In [16]:
temp = pd.DataFrame(data=score_results)
x = temp.loc[:, temp.columns != 'Features'].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
temp = pd.DataFrame(x_scaled)

mean_results =  temp.mean(axis=1)
score_results['avg_scores'] = mean_results

choice = widgets.Dropdown(
    options=['Pearson', 'Spearmanr', 'Kendall', 'Fisher', 'Variance TH', 'Random Forest Importance', 'Results Mean'],
    value='Pearson',
    description='Method:',
    disabled=False,
)

options = {'Pearson' : 'pearson',
           'Spearmanr' : 'spearmanr',
           'Kendall' : 'kendall',
           'Fisher' : 'fisher',
           'Variance TH' : 'variance_threshold',
           'Random Forest Importance' : 'RF_importance',
           'Results Mean' : 'avg_scores'
           }


norm = widgets.Checkbox(
    value=True,
    description='Results normalized',
    disabled=False,
    indent=True
)

scale = widgets.RadioButtons(
    options=['Regular', 'Logaritmic'],
    description='Scale:',
    disabled=False,
)
order = widgets.RadioButtons(
    options=['Labels', 'Scores'],
    description='Order by:',
    disabled=False
)


def barPlot_manager(change_scale, change_order, method, normalized):
    res = score_results[["Features", options[method]]]
    res.rename(columns = {options[method]:'Scores'}, inplace = True)
    df = pd.DataFrame(data=res)
    if (change_scale == 'Logaritmic'):
        if (change_order == 'Scores'):
            df = df.sort_values(by='Scores', ascending=False)
            if normalized:
                m.show_bar_log(df['Features'], m.NormalizeData1D(df['Scores']), method)
            else:
                m.show_bar_log(df['Features'], df['Scores'], method)
            return
        else:
            if normalized:
                m.show_bar_log(df['Features'], m.NormalizeData1D(df['Scores']), method)
            else:
                m.show_bar_log(df['Features'], df['Scores'], method)
            return
    else:
        if (change_order == 'Scores'):
            df = df.sort_values(by='Scores', ascending=False)
            if normalized:
                m.show_bar(df['Features'], m.NormalizeData1D(df['Scores']), method)
            else:
                m.show_bar(df['Features'], df['Scores'], method)
            return
        else:
            if normalized:
                m.show_bar(df['Features'], m.NormalizeData1D(df['Scores']), method)
            else:
                m.show_bar(df['Features'], df['Scores'], method)
        return


container1 = widgets.VBox([scale, order])
container2 = widgets.VBox([choice, norm])
ui = widgets.HBox([container2, container1])
out = widgets.interactive_output(barPlot_manager, {'method':choice, 'change_scale': scale, 'change_order': order, 'normalized': norm})
display(ui, out)









Output()